In [ ]:
import os

import numpy as np

import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display

import matplotlib.pyplot as plt
%matplotlib notebook

import imars3d
import tomopy
import timeit

from imars3d.backend.dataio.data import load_data, _get_filelist_by_dir
from imars3d.backend.morph.crop import crop, detect_bounds
from imars3d.backend.corrections.gamma_filter import gamma_filter
from imars3d.backend.preparation.normalization import normalization
from imars3d.backend.diagnostics import tilt
from imars3d.backend.diagnostics.rotation import find_rotation_center

In [ ]:
ncore = 10

In [ ]:
ct_dir = "/HFIR/CG1D/IPTS-29298/raw/ct_scans/2022_09_29_sample3"
assert os.path.exists(ct_dir)

ob_dir = "/HFIR/CG1D/IPTS-29298/raw/ob/2022_09_30"
assert os.path.exists(ob_dir)

dc_dir = "/HFIR/CG1D/IPTS-29298/raw/df/2022_09_29"
assert os.path.exists(dc_dir)

# Load data 

In [ ]:
#%%timeit
# proj_raw, ob_raw, dc_raw, rot_angles = load_data(ct_dir=ct_dir,
#                                    ob_dir=ob_dir,
#                                    dc_dir=dc_dir,
#                                    ct_fnmatch="*.tiff",
#                                    ob_fnmatch="*.tiff",
#                                    dc_fnmatch="*.tiff")
# proj_min = np.min(proj_raw, axis=0)

In [ ]:
t0 = timeit.default_timer()
proj_raw, ob_raw, dc_raw, rot_angles = load_data(ct_dir=ct_dir,
                                   ob_dir=ob_dir,
                                   dc_dir=dc_dir,
                                   ct_fnmatch="*.tiff",
                                   ob_fnmatch="*.tiff",
                                   dc_fnmatch="*.tiff")
proj_min = np.min(proj_raw, axis=0)
t1 = timeit.default_timer()
print(f"time: {t1-t0} s")

In [ ]:
fig, (ax0, ax1, ax2) = plt.subplots(nrows=3, ncols=1, figsize=(5,9))
proj_min = np.min(proj_raw, axis=0)
ob_min = np.min(ob_raw, axis=0)
dc_max = np.max(dc_raw, axis=0)

plt0 = ax0.imshow(proj_min)
fig.colorbar(plt0, ax=ax0)
ax0.set_title("np.min(proj_raw)")

plt1 = ax1.imshow(ob_min)
fig.colorbar(plt1, ax=ax1)
ax1.set_title("np.min(ob_raw)")

plt2 = ax2.imshow(dc_max)
fig.colorbar(plt2, ax=ax2)
ax2.set_title("np.min(dc_raw)")

fig.tight_layout()

# Crop 

In [ ]:
crop_region = [600, 1350, 100, 1950]    # [left, right, top, bottom]

In [ ]:
proj_crop = crop(arrays=proj_raw,
         crop_limit=crop_region)
ob_crop = crop(arrays=ob_raw,
         crop_limit=crop_region)
dc_crop = crop(arrays=dc_raw,
         crop_limit=crop_region)
proj_crop_min = crop(arrays=proj_min,
                    crop_limit=crop_region)

In [ ]:
plt.figure()
plt.imshow(proj_crop_min)
plt.colorbar()

# gamma filtering 

In [ ]:
t0 = timeit.default_timer()
proj_gamma = gamma_filter(arrays=proj_crop, 
                        selective_median_filter=False, 
                        diff_tomopy=20, 
                        max_workers=48, 
                        median_kernel=3)
ob_gamma = ob_crop
dc_gamma = dc_crop
t1 = timeit.default_timer()
print(f"time: {t1-t0} s")

In [ ]:
proj_gamma_min = np.min(proj_gamma, axis=0)
plt.figure(num="Gamma filtering")
plt.imshow(proj_gamma_min)
plt.colorbar()

# normalization 

While I'm still waiting for imars3D to fix the normalization

In [ ]:
# %%time
# ct_normalized = normalization(arrays=proj_gamma,
#                               flats=ob_gamma,
#                               darks=dc_gamma)

In [ ]:
my_ob = np.median(ob_gamma, axis=0)
my_dc = np.median(dc_gamma, axis=0)

proj_norm = []
for ct in proj_gamma:
    proj_norm.append(np.true_divide(ct-my_dc, my_ob-my_dc))

proj_norm = np.asarray(proj_norm)

In [ ]:
proj_norm_min = np.min(proj_norm, axis=0)
plt.figure(num="Normalization")
plt.imshow(proj_norm_min)
plt.colorbar()

# beam fluctuation 

Using normalize_roi because part of the rotation stage is in the way and does not allow to use normalize_bg

In [ ]:
bg_region = [5, 100, 250, 1100]  #  [left, right, top, bottom]

In [ ]:
proj_norm_bg = proj_norm[:, bg_region[2]: bg_region[3],
                            bg_region[0]: bg_region[1]]
proj_norm_bg_min = np.min(proj_norm_bg, axis=0)
plt.figure()
plt.imshow(proj_norm_bg_min)
plt.colorbar()

In [ ]:
roi = [bg_region[2], bg_region[0],
       bg_region[3], bg_region[1]]

In [ ]:
t0 = timeit.default_timer()
proj_norm_beam_fluctuation = tomopy.prep.normalize.normalize_roi(proj_norm,
                                               roi=roi,
                                               ncore=ncore)
t1 = timeit.default_timer()
print(f"time: {t1-t0} s")

In [ ]:
fig, (ax0, ax1, ax2) = plt.subplots(nrows=3, ncols=1,
                               num="Beam fluctuation",
                               figsize=(5,15))

# before beam fluctuation
proj_norm_min = np.min(proj_norm, axis=0)
fig0 = ax0.imshow(proj_norm_min)
ax0.set_title("before")
plt.colorbar(fig0, ax=ax0)

# after beam fluctuation
proj_norm_beam_fluctuation_min = np.min(proj_norm_beam_fluctuation, axis=0)
fig1 = ax1.imshow(proj_norm_beam_fluctuation_min)
ax1.set_title("after")
plt.colorbar(fig1, ax=ax1)

# difference
diff = proj_norm_beam_fluctuation[0] - proj_norm[0]
fig2 = ax2.imshow(diff)
ax2.set_title("difference")
plt.colorbar(fig2, ax=ax2)

# transmission to attenuation 

In [ ]:
proj_mlog = tomopy.minus_log(proj_norm_beam_fluctuation)

In [ ]:
proj_mlog_min = np.min(proj_mlog, axis=0)
plt.figure()
plt.imshow(proj_mlog_min)
plt.colorbar()

# Tilt correction

Manually calculating the `atol` parameter

In [ ]:
delta_angle = rot_angles[1] - rot_angles[0]
mean_delta_angle = np.mean([y-x for (x,y) in zip(rot_angles[:-1], rot_angles[1:])])

list_180_deg_pairs_idx = tilt.find_180_deg_pairs_idx(angles=rot_angles,
                                                    atol=mean_delta_angle)

checking that the 180degrees file found is correct

In [ ]:
from imars3d.backend.dataio.data import _get_filelist_by_dir

In [ ]:
index_0_degree = list_180_deg_pairs_idx[0][0]
index_180_degree = list_180_deg_pairs_idx[1][0]


list_ct_files, list_ob_files, list_dc_files = _get_filelist_by_dir(ct_dir=ct_dir,
                                                                  ob_dir=ob_dir)

file_0_degree = list_ct_files[index_0_degree]
file_180_degree = list_ct_files[index_180_degree]
file_180_next_degree = list_ct_files[index_180_degree+1]

print(f"file 0 degrees: {os.path.basename(file_0_degree)}")
print(f"file 180 degrees: {os.path.basename(file_180_degree)}")
print(f"next file is: {os.path.basename(file_180_next_degree)}")

Calculate tilt correction value

In [ ]:
tilt_angle = tilt.calculate_tilt(image0=proj_mlog[index_0_degree],
                                image180=proj_mlog[index_180_degree])
print(f"tilt_angle: {tilt_angle}")

apply the titl correction

In [ ]:
proj_tilt_corrected = tilt.apply_tilt_correction(arrays=proj_mlog,
                                                tilt=tilt_angle.x)

# Rotation center 

In [ ]:
t0 = timeit.default_timer()
rot_center = find_rotation_center(arrays=proj_crop,
                                 angles=rot_angles,
                                 in_degrees=True,
                                 atol_deg=mean_delta_angle
                                 )
t1 = timeit.default_timer()
print(f"time: {t1-t0} s")

In [ ]:
print(f"{rot_center=}")